In [ ]:
#%pip install openai pydantic
%load_ext autoreload
%autoreload 2
import os
from tqdm.notebook import tqdm
from mani_skill.examples.objaverse_handler import SpokDatasetBuilder

In [ ]:
N_samples = 20_000
spock_dataset = SpokDatasetBuilder(maximum_objects=N_samples)
print("dataset size", len(spock_dataset))

In [ ]:
from PIL import  Image
uids = sorted(spock_dataset.sample_uuids(N_samples))

for uid in tqdm(uids):
    print(spock_dataset.get_condensed_gpt_description(uid))
    image_files = sorted( list((spock_dataset.spok_models_path / uid / "blender_renders" ).iterdir()))
    display(Image.open(image_files[0]))
    break


In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModel
import torch

model = AutoModel.from_pretrained("google/siglip-base-patch16-224")
processor = AutoProcessor.from_pretrained("google/siglip-base-patch16-224")

In [ ]:
import json
# get files with description
have_description = []
for uid in tqdm(uids):
    description_path = spock_dataset.spok_models_path / uid / "chatgpt_description.json"
    if description_path.is_file():
        have_description.append(uid)

# get files with scored descriptions
scored_descr = spock_dataset.spok_root_path / "clip_description.jsonl"
scored_uids = []

print(scored_descr)
if not scored_descr.is_file():
    open(scored_descr, 'a').close()
else:
    with open(scored_descr, "r") as f_obj:
        for line in f_obj:
            data = json.loads(line)  # Parse each line as a JSON object
            if "uid" in data:  # Check if 'uid' key exists
                scored_uids.append(data["uid"])

print("descrd uids", len(have_description))
print("scored uids", len(scored_uids))
to_score_uids = set(have_description) - set(scored_uids)
print("to score:", len(to_score_uids))

In [ ]:
import json
new_entries = []

for uid in tqdm(to_score_uids):
    image_files = sorted( list((spock_dataset.spok_models_path / uid / "blender_renders" ).iterdir()))
    image = Image.open(image_files[0])
    descriptions = spock_dataset.get_condensed_gpt_description(uid).values()
    descriptions = [x.lower() for x in descriptions if type(x) is str]

    descr_texts = [f'This is a picture of a {label}.' for label in descriptions]
    extra_texts = ["grayscale image", "cartoon low-poly model"]

    texts = descr_texts + extra_texts
    inputs = processor(text=texts, images=image, padding="max_length", return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image[0]

    entry = {"uid":uid, "description": {}, "quality": {}}
    for label, logit in zip(descriptions + extra_texts, logits_per_image):
        is_descr = label not in extra_texts
        if is_descr:
            entry["description"][label] = round(float(logit), 2)
        else:
            entry["quality"][label] = round(float(logit), 2)
    
    new_entries.append(entry)
    if len(new_entries) > 50:
        display(image)
        print(entry)
        
        with open(scored_descr, "a") as f_obj:
            for entry in new_entries:
                f_obj.write(json.dumps(entry) + "\n")
        new_entries = []

In [ ]:
with open(scored_descr, "a") as f_obj:
    for entry in new_entries:
        f_obj.write(json.dumps(entry) + "\n")
